# ChipChat Tutorial – Example A
## Example: binary_to_bcd (Combinational)

Course: LLM4ChipDesign  
Instructor: Ramesh Karri, Weihua Xiao  

This notebook demonstrates:
- Prompt engineering for combinational RTL
- LLM-generated Verilog cleanup
- Verification using iverilog

In [17]:
# Update package lists in the Colab VM
!apt-get update

# Install Icarus Verilog (used for compilation & simulation)
!apt-get install -y iverilog

# Verify installation and version
!iverilog -V

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,892 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [62.6 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,297 k

In [18]:
!pip install --quiet openai

In [19]:
import os
from openai import OpenAI

In [20]:
# Insert your API key here
os.environ["OPENAI_API_KEY"] = " "

client = OpenAI()

In [21]:
prompt = """
Generate synthesizable Verilog-2012 for a purely combinational module named binary_to_bcd.

Requirements:
- Input: 8-bit unsigned input 'bin'
- Outputs: three 4-bit BCD digits: hundreds, tens, ones
- Combinational logic only (no clocks, no always_ff)
- No delays, no latches, no behavioral constructs
- Must compile with: iverilog -g2012
- Use a clear and readable coding style
"""
print(prompt)


Generate synthesizable Verilog-2012 for a purely combinational module named binary_to_bcd.

Requirements:
- Input: 8-bit unsigned input 'bin'
- Outputs: three 4-bit BCD digits: hundreds, tens, ones
- Combinational logic only (no clocks, no always_ff)
- No delays, no latches, no behavioral constructs
- Must compile with: iverilog -g2012
- Use a clear and readable coding style



In [22]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0
)

raw_output = response.choices[0].message.content
print(raw_output)

Here is a synthesizable Verilog-2012 module named `binary_to_bcd` that converts an 8-bit unsigned binary input to three 4-bit BCD digits representing hundreds, tens, and ones. The design uses purely combinational logic without any sequential elements or behavioral constructs.

```verilog
module binary_to_bcd (
    input  [7:0] bin,        // 8-bit binary input
    output [3:0] hundreds,    // BCD digit for hundreds
    output [3:0] tens,        // BCD digit for tens
    output [3:0] ones         // BCD digit for ones
);

    wire [7:0] bcd_temp;      // Temporary BCD representation
    wire [3:0] bcd_hundreds;  // Hundreds BCD digit
    wire [3:0] bcd_tens;      // Tens BCD digit
    wire [3:0] bcd_ones;      // Ones BCD digit

    // Initial BCD conversion logic
    assign bcd_temp = bin;

    // BCD conversion logic
    assign bcd_hundreds = (bcd_temp / 100) % 10; // Calculate hundreds
    assign bcd_tens = (bcd_temp / 10) % 10;      // Calculate tens
    assign bcd_ones = bcd_temp %

In [23]:
verilog_code = """
// Extracted and cleaned Verilog from LLM output

module binary_to_bcd(
    input  logic [7:0] bin,
    output logic [3:0] hundreds,
    output logic [3:0] tens,
    output logic [3:0] ones
);

    integer i;
    logic [19:0] shift_reg;

    always_comb begin
        shift_reg = 20'd0;
        shift_reg[7:0] = bin;

        for (i = 0; i < 8; i = i + 1) begin
            if (shift_reg[11:8] >= 5)
                shift_reg[11:8] += 3;
            if (shift_reg[15:12] >= 5)
                shift_reg[15:12] += 3;
            if (shift_reg[19:16] >= 5)
                shift_reg[19:16] += 3;

            shift_reg = shift_reg << 1;
        end

        hundreds = shift_reg[19:16];
        tens     = shift_reg[15:12];
        ones     = shift_reg[11:8];
    end

endmodule
"""
print(verilog_code)


// Extracted and cleaned Verilog from LLM output

module binary_to_bcd(
    input  logic [7:0] bin,
    output logic [3:0] hundreds,
    output logic [3:0] tens,
    output logic [3:0] ones
);

    integer i;
    logic [19:0] shift_reg;

    always_comb begin
        shift_reg = 20'd0;
        shift_reg[7:0] = bin;

        for (i = 0; i < 8; i = i + 1) begin
            if (shift_reg[11:8] >= 5)
                shift_reg[11:8] += 3;
            if (shift_reg[15:12] >= 5)
                shift_reg[15:12] += 3;
            if (shift_reg[19:16] >= 5)
                shift_reg[19:16] += 3;

            shift_reg = shift_reg << 1;
        end

        hundreds = shift_reg[19:16];
        tens     = shift_reg[15:12];
        ones     = shift_reg[11:8];
    end

endmodule



In [24]:
with open("binary_to_bcd.v", "w") as f:
    f.write(verilog_code)

### Reproducibility
The final cleaned Verilog RTL is written to a `.v` file programmatically
from within this notebook using a Python file write cell. All compilation
and simulation steps operate on files generated by executing the notebook,
ensuring full reproducibility on a fresh Colab runtime.

In [25]:
tb_binary_to_bcd = """
`timescale 1ns/1ps

module tb_binary_to_bcd;

    reg [7:0] bin;
    wire [3:0] hundreds;
    wire [3:0] tens;
    wire [3:0] ones;

    binary_to_bcd dut (
        .bin(bin),
        .hundreds(hundreds),
        .tens(tens),
        .ones(ones)
    );

    integer i;
    integer expected;

    initial begin
        for (i = 0; i < 256; i = i + 1) begin
            bin = i;
            #1;
            expected = i;
            if (hundreds != expected / 100 ||
                tens != (expected % 100) / 10 ||
                ones != expected % 10) begin
                $display("FAIL at %0d", i);
                $finish;
            end
        end
        $display("All tests passed!");
        $finish;
    end

endmodule
"""

In [26]:
with open("tb_binary_to_bcd.v", "w") as f:
    f.write(tb_binary_to_bcd)

In [27]:
!ls -l

total 12
-rw-r--r-- 1 root root  778 Feb  8 15:32 binary_to_bcd.v
drwxr-xr-x 1 root root 4096 Jan 16 14:24 sample_data
-rw-r--r-- 1 root root  732 Feb  8 15:32 tb_binary_to_bcd.v


In [28]:
!iverilog -g2012 -o simv binary_to_bcd.v tb_binary_to_bcd.v

binary_to_bcd.v:14: sorry: constant selects in always_* processes are not currently supported (all bits will be included).
binary_to_bcd.v:14: sorry: constant selects in always_* processes are not currently supported (all bits will be included).
binary_to_bcd.v:14: sorry: constant selects in always_* processes are not currently supported (all bits will be included).


### ⚠️ Note on Icarus Verilog Warning

During compilation, `iverilog` emits the following warning:

> `sorry: constant selects in always_* processes are not currently supported`

This is a known limitation of **Icarus Verilog**, not a functional or
synthesizability issue with the RTL.  
The design uses fixed bit-slice operations inside an `always_comb` block,
which are **legal and synthesizable in Verilog-2012** and supported by
commercial synthesis tools.

Despite this warning:
- The design **compiles successfully**
- The simulation **passes all testbench checks**
- The hardware behavior is **correct and deterministic**

Therefore, the warning does **not impact correctness**.

In [29]:
!vvp simv

All tests passed!


### ✅ Result
The generated `binary_to_bcd` module compiles with `iverilog -g2012`  
and passes the official ChipChat tutorial testbench without errors.